In [9]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Mini-config
IMG_SIZE = (224, 224)
BATCH_SIZE = 4  # Small batch for testing
EPOCHS = 3  # Just 3 epochs for validation

In [12]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/data/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/data/validation',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)



Found 1792 images belonging to 2 classes.
Found 641 images belonging to 2 classes.


In [4]:
# Save class_indices for prediction use
os.makedirs('models', exist_ok=True)  # Make sure 'models' directory exists
with open('models/class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

In [5]:
# Model setup (frozen ResNet50 + 1 dense layer)
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(train_generator.num_classes, activation='softmax')(x)  # Match to number of classes
model = Model(base_model.input, x)




94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Test training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 5), output.shape=(None, 2)

In [ ]:
# Quick visualization
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.savefig('models/training_plot.png')
plt.show()


In [ ]:
# Save trained model
model.save('models/final_model.h5')

In [ ]:
print("\n🔥 Training complete!")
print("✅ Model saved as 'models/final_model.h5'")
print("✅ Class indices saved as 'models/class_indices.json'")
print("✅ Accuracy plot saved as 'models/training_plot.png'")
